<a href="https://colab.research.google.com/github/t3noch/colabs/blob/main/Copia_de_10_CoT_peliculas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 1️⃣ Instalar librerías necesarias en Google Colab
!pip install scikit-surprise pandas matplotlib

In [ ]:
# 📌 2️⃣ Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [ ]:
!wget --no-check-certificate -O ml-100k.zip https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q ml-100k.zip


In [ ]:
# 📌 3️⃣ Descargar y cargar el dataset MovieLens 100k
ratings = ___.read_csv("ml-100k/u.data", sep="\t", names=["id_usuario", "id_pelicula", "rating", "timestamp"])
peliculas = ___.___("ml-100k/u.item", sep="|", encoding="latin-1", usecols=[0, 1], names=["id_pelicula", "titulo"])



In [ ]:
# 📌 5️⃣ Preparar los datos para Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[["id_usuario", "id_pelicula", "rating"]], reader)

# 📌 6️⃣ Dividir datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
# 📌 7️⃣ Entrenar modelo SVD
modelo = SVD()
modelo.fit(trainset)

# 📌 8️⃣ Función para recomendar películas con Chain of Thought (CoT)
def recomendar_pelicula(id_usuario):
    # Obtener películas vistas por el usuario
    peliculas_vistas = ratings[ratings["id_usuario"] == id_usuario]["id_pelicula"].tolist()
    peliculas_no_vistas = peliculas[~peliculas["id_pelicula"].isin(peliculas_vistas)]

    # Predecir calificación para cada película no vista
    predicciones = []
    for _, row in peliculas_no_vistas.iterrows():
        pred = modelo.predict(id_usuario, row["id_pelicula"])
        predicciones.append((row["id_pelicula"], pred.est))

    # Ordenar por mejor predicción
    predicciones.sort(key=lambda x: x[1], reverse=True)

    # Tomar la mejor película recomendada
    mejor_pelicula_id, puntuacion = predicciones[0]
    titulo_recomendado = peliculas[peliculas["id_pelicula"] == mejor_pelicula_id]["titulo"].values[0]

    # 📌 9️⃣ Generar explicación CoT para la recomendación
    peliculas_favoritas = peliculas[peliculas["id_pelicula"].isin(peliculas_vistas)].head(5)

    print("\n🔍 Análisis de usuario:")
    print(f"🎬 Películas favoritas: {', '.join(peliculas_favoritas['titulo'].values)}")

    print("\n📝 Explicación paso a paso (Chain of Thought):")
    print(f"1️⃣ Has visto muchas películas con alto puntaje.")
    print(f"2️⃣ '{titulo_recomendado}' ha sido altamente valorada por usuarios similares.")
    print(f"3️⃣ Basado en el modelo, se estima que te gustaría con {puntuacion:.2f}/5.")
    print(f"✅ Recomendación final: ¡Mira '{titulo_recomendado}'!")

In [ ]:
# 📌 🔟 Probar recomendación para un usuario
recomendar_pelicula(2)